In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import pandas as pd
import spacy
from spacy.training.example import Example
import random

In [40]:
df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/AI-Powered-Document-Analysis-System/data/train/merged_train_sentences.csv")


In [41]:
df.head()

,Sentences,Category
0,The stock market experienced a significant ris...,financial
1,Investing in mutual funds can be a good way to...,financial
2,The Federal Reserve announced an increase in i...,financial
3,Cryptocurrencies like Bitcoin are highly volat...,financial
4,It's important to have an emergency fund cover...,financial


In [42]:
df

,Sentences,Category
0,The stock market experienced a significant ris...,financial
1,Investing in mutual funds can be a good way to...,financial
2,The Federal Reserve announced an increase in i...,financial
3,Cryptocurrencies like Bitcoin are highly volat...,financial
4,It's important to have an emergency fund cover...,financial
...,...,...
3406,Legal prohibitions exist against insider tradi...,legal
3407,Surveillance laws regulate the use of electron...,legal
3408,Legal instruments such as affidavits serve as ...,legal
3409,Public safety laws govern the storage and hand...,legal


In [43]:
import pandas as pd
import random
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [44]:
df.columns = df.columns.str.replace('ï»¿', '')


In [45]:
category_mapping = {'healthcare': 0, 'financial': 1, 'legal': 2}
df['Category'] = df['Category'].map(category_mapping)

In [46]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)


In [47]:
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [48]:
train_texts = train_data['Sentences'].tolist()
train_labels = train_data['Category'].tolist()

test_texts = test_data['Sentences'].tolist()
test_labels = test_data['Category'].tolist()

In [49]:
nlp = spacy.blank("en")

In [50]:
if "textcat" not in nlp.pipe_names:
    textcat = nlp.add_pipe("textcat", last=True)
else:
    textcat = nlp.get_pipe("textcat")

In [51]:
categories = [0, 1, 2]
for category in categories:
    textcat.add_label(str(category))

In [52]:
train_data_spacy = []
for text, label in zip(train_texts, train_labels):
    cats = {str(cat): label == cat for cat in categories}
    train_data_spacy.append((text, {"cats": cats}))

In [53]:
n_iter = 30
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]

In [54]:
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for i in range(n_iter):
        losses = {}
        random.shuffle(train_data_spacy)
        batches = minibatch(train_data_spacy, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            examples = [Example.from_dict(nlp.make_doc(text), annotation) for text, annotation in zip(texts, annotations)]
            nlp.update(examples, sgd=optimizer, drop=0.2, losses=losses)
        print(f"Iteration {i+1}: Losses {losses}")

Iteration 1: Losses {'textcat': 45.439850663307766}
Iteration 2: Losses {'textcat': 5.769544692802709}
Iteration 3: Losses {'textcat': 2.6037065353552773}
Iteration 4: Losses {'textcat': 1.7716935172776545}
Iteration 5: Losses {'textcat': 1.1404457657947433}
Iteration 6: Losses {'textcat': 0.4944467379804975}
Iteration 7: Losses {'textcat': 2.509021899726248}
Iteration 8: Losses {'textcat': 2.336087345289811}
Iteration 9: Losses {'textcat': 1.200294893164926}
Iteration 10: Losses {'textcat': 1.4051199005280177}
Iteration 11: Losses {'textcat': 0.6373596557699648}
Iteration 12: Losses {'textcat': 0.7424464828497845}
Iteration 13: Losses {'textcat': 0.5558122462159466}
Iteration 14: Losses {'textcat': 1.658540104912221}
Iteration 15: Losses {'textcat': 1.32218573236483}
Iteration 16: Losses {'textcat': 0.7194051554568665}
Iteration 17: Losses {'textcat': 1.0657581224191082}
Iteration 18: Losses {'textcat': 0.3428566410936601}
Iteration 19: Losses {'textcat': 0.18483049250323003}
Iteratio

In [55]:
output_dir = r"/content/drive/MyDrive/Colab Notebooks/AI-Powered-Document-Analysis-System/model_training"
nlp.to_disk(output_dir)
print(f"Model saved to {output_dir}")


Model saved to /content/drive/MyDrive/Colab Notebooks/AI-Powered-Document-Analysis-System/model_training


In [56]:
nlp = spacy.load(output_dir)


In [57]:
predicted_labels = []
for text in test_texts:
    doc = nlp(text)
    predicted_label = max(doc.cats, key=doc.cats.get)  # Get the label with the highest confidence
    predicted_labels.append(int(predicted_label))

In [58]:
report = classification_report(test_labels, predicted_labels, target_names=['healthcare', 'financial', 'legal'])
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

  healthcare       0.96      0.98      0.97       198
   financial       0.99      0.96      0.97       250
       legal       0.96      0.97      0.96       235

    accuracy                           0.97       683
   macro avg       0.97      0.97      0.97       683
weighted avg       0.97      0.97      0.97       683



In [59]:
conf_matrix = confusion_matrix(test_labels, predicted_labels, labels=categories)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[195   0   3]
 [  4 240   6]
 [  5   3 227]]


In [60]:
test_csv_path = r'/content/drive/MyDrive/Colab Notebooks/AI-Powered-Document-Analysis-System/data/train/merged_train_sentences.csv'  # Update with actual test CSV file path
test_df = pd.read_csv(test_csv_path)

In [61]:
test_texts = test_df['Sentences'].tolist()
test_labels = test_df['Category'].map(category_mapping).tolist()

In [62]:
predicted_labels = []
for text in test_texts:
    doc = nlp(text)
    predicted_label = max(doc.cats, key=doc.cats.get)  # Get the label with the highest confidence
    predicted_labels.append(int(predicted_label))

In [63]:
report = classification_report(test_labels, predicted_labels, target_names=['healthcare', 'financial', 'legal'])
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

  healthcare       0.99      1.00      0.99      1121
   financial       1.00      0.99      0.99      1109
       legal       0.99      0.99      0.99      1181

    accuracy                           0.99      3411
   macro avg       0.99      0.99      0.99      3411
weighted avg       0.99      0.99      0.99      3411



In [64]:
conf_matrix = confusion_matrix(test_labels, predicted_labels, labels=categories)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[1118    0    3]
 [   6 1097    6]
 [   5    3 1173]]


In [65]:
test_csv_path2 = r'/content/drive/MyDrive/Colab Notebooks/AI-Powered-Document-Analysis-System/data/test/output.csv'  # Update with actual test CSV file path
test_df2 = pd.read_csv(test_csv_path2)

In [66]:
test_texts = test_df['Sentences'].tolist()
test_labels = test_df['Category'].map(category_mapping).tolist()

In [67]:
predicted_labels = []
for text in test_texts:
    doc = nlp(text)
    predicted_label = max(doc.cats, key=doc.cats.get)  # Get the label with the highest confidence
    predicted_labels.append(int(predicted_label))

In [68]:
report = classification_report(test_labels, predicted_labels, target_names=['healthcare', 'financial', 'legal'])
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

  healthcare       0.99      1.00      0.99      1121
   financial       1.00      0.99      0.99      1109
       legal       0.99      0.99      0.99      1181

    accuracy                           0.99      3411
   macro avg       0.99      0.99      0.99      3411
weighted avg       0.99      0.99      0.99      3411

